# Create small difficult corpus
 Notebook to create a small, yet difficult corpus, for quick experimentation. In many cases, there are a lot of documents to search through, and indexing these documents with various methods can take a very long time. To avoid lenghty experiments, yet making sure that the corpus oen searches through is "hard", this notebook creates a small corpus (e.g. 10000 documents) that contain:
  * all goldstandard documents
  * top 100 documents per query (obtained with some method - could be the model itself, another model, or bm25)
  * other random documents to pad to the desired size

In [3]:
from docuverse.utils import open_stream, read_config_file
from docuverse.engines.search_engine import SearchEngine
import os
import orjson
from  tqdm import tqdm_notebook as tqdm

In [4]:
data_dir="/home/raduf/sandbox2/docuverse/benchmark/ibm_search"
query_file = os.path.join(data_dir, "question_test_config.json")
corpus_file = os.path.join(data_dir, "corpus.jsonl.bz2")
query_config = read_config_file(query_file)
query_json = query_config['question_file']
goldstandard = query_config['goldstandard_file']
top100 = "/home/raduf/sandbox2/docuverse/output/"

In [5]:
config_file = os.path.join("/home/raduf/sandbox2/docuverse/experiments/unified_search/ibmsw_milvus_dense.snowlake-m.test.yaml")
engine = SearchEngine(config_file)

Running on the gpus:  ['NVIDIA GeForce RTX 4090']


/home/raduf/miniforge3/envs/docuverse/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


=== done initializing model


In [6]:
for i, line in enumerate(open_stream(corpus_file)):
    # print(line)
    print(orjson.loads(line))
    if i==10:
        break

{'title': 'aclput Command', '_id': '1', 'text': 'aclput Command Purpose Sets the access control information of a file. Syntax aclput [ -i inAclFile ] [ -R ] [ -t acl_type ] [ -v ]FileObject Description The aclput command sets the access control information of the file object that is specified by the FileObject parameter. The command reads standard input for the access control information, unless you specify the -i flag. Note: If you are reading from standard input your entries must match the expected format of the access control information or you get an error message. Use the Ctrl-D key sequence to complete the session. Access Control List Access Control Lists form the core of protection for file system objects. Each file system object is uniquely associated with one piece of data, called ACL that defines the access rights to the object. ACL might consist of multiple Access Control Entries (ACEs), each defining one particular set of access rights for a user. Typically, ACE consists of

In [2]:
corpus = [orjson.loads(line) for line in tqdm(open_stream(corpus_file))]

NameError: name 'tqdm' is not defined

In [ ]:
documents = {}
for c in corpus:
    documents[c['_id']] = c

In [30]:
documents['1']

{'title': 'aclput Command',
 '_id': '1',
 'text': 'aclput Command Purpose Sets the access control information of a file. Syntax aclput [ -i inAclFile ] [ -R ] [ -t acl_type ] [ -v ]FileObject Description The aclput command sets the access control information of the file object that is specified by the FileObject parameter. The command reads standard input for the access control information, unless you specify the -i flag. Note: If you are reading from standard input your entries must match the expected format of the access control information or you get an error message. Use the Ctrl-D key sequence to complete the session. Access Control List Access Control Lists form the core of protection for file system objects. Each file system object is uniquely associated with one piece of data, called ACL that defines the access rights to the object. ACL might consist of multiple Access Control Entries (ACEs), each defining one particular set of access rights for a user. Typically, ACE consists 

In [7]:
queries = engine.read_questions(engine.config.input_queries)

Reading benchmark/ibm_search/test-queries.jsonl:: 402it [00:00, 34833.39it/s]


In [19]:
relevant = {}
for q in queries:
    for r in q['relevant']:
        relevant[r] = 1

In [21]:
answers = engine.read_cache_file(extension=".retrieve.pkl.bz2")

File /home/raduf/sandbox2/docuverse/experiments/unified_search/cache/ibmsw-full-milvus-dense-snowflake-m-512-100-574485.retrieve.pkl.bz2 exists, read?
Reading cached search results from /home/raduf/sandbox2/docuverse/experiments/unified_search/cache/ibmsw-full-milvus-dense-snowflake-m-512-100-574485.retrieve.pkl.bz2


In [23]:
ansdocs = {}
for a in answers:
    for result in a['results']:
        ansdocs[result['doc_id']] = 1

TypeError: list indices must be integers or slices, not str

In [24]:
for a in answers[0]:
    print(a)